In [1]:
import pyodbc
import pandas as pd

In [ ]:
pd.set_option("display.expand_frame_repr", None)

In [2]:
pd.reset_option("display.width", None)

In [3]:
driver = '{SQL Server}'
server = 'DESKTOP-41IVAA9'
database = 'Customers'
username = 'sa'
password = '1'
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [4]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

In [5]:
def sorgu(query, select=True):
    cursor = connection.cursor()
    cursor.execute(query)
    if select:
        rows = cursor.fetchall()
        rows = [list(row) for row in rows]
        columns = [column[0] for column in cursor.description]
        return pd.DataFrame(rows, columns=columns)

In [ ]:
df = sorgu("SELECT * FROM CUSTOMERS")
df

#### SORU 6

In [6]:
# -- SORU 6
df = sorgu("SELECT * FROM CUSTOMERS WHERE NAMESURNAME LIKE 'A%'")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP
0,6,Ahmet İNCİKAPI,6722155596,E,a_incikapi@miuul.com,1991-05-28,53,225,(532)2414618,(538)8459085,20-35 YAŞ
1,7,Arif TEMELOĞLU,43691911318,E,a_temeloglu@miuul.com,1967-12-11,40,638,(554)5504524,(534)6379277,56-65 YAŞ
2,9,Ali Eymen DEVE,80011834707,E,a_eymen@miuul.com,1985-01-23,39,463,(533)8082176,(538)4811026,36-45 YAŞ
3,88,Ayhan ÖZÇİL,3328037338,E,a_ozcil@miuul.com,1963-11-08,10,114,(543)2007274,(534)1062665,56-65 YAŞ
4,89,Azad ÖNÜR,65514513990,E,a_onvr@miuul.com,1989-03-23,55,722,(532)7551426,(542)2487070,20-35 YAŞ
...,...,...,...,...,...,...,...,...,...,...,...
73,924,Ali CENGİARSLAN,37707906321,E,a_cengiarslan@miuul.com,1988-01-19,27,728,(542)6905828,(533)3111924,20-35 YAŞ
74,945,Aslıhan BİLALOGLU,78549819278,K,a_bilaloglu@miuul.com,1968-09-20,15,249,(535)1451111,(541)5962564,46-55 YAŞ
75,975,Aras KORKMAN,48942375673,E,a_korkman@miuul.com,1950-07-12,15,249,(553)2204286,(533)3091497,65+ YAŞ
76,985,Ayhan SAYIT,30417025079,E,a_sayit@miuul.com,1982-05-24,37,720,(538)6749244,(541)6986592,36-45 YAŞ


#### SORU 7

In [7]:
# -- SORU 7
df = sorgu("""
SELECT * 
FROM CUSTOMERS 
WHERE NAMESURNAME LIKE 'A%'
AND GENDER = 'E'""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP
0,6,Ahmet İNCİKAPI,6722155596,E,a_incikapi@miuul.com,1991-05-28,53,225,(532)2414618,(538)8459085,20-35 YAŞ
1,7,Arif TEMELOĞLU,43691911318,E,a_temeloglu@miuul.com,1967-12-11,40,638,(554)5504524,(534)6379277,56-65 YAŞ
2,9,Ali Eymen DEVE,80011834707,E,a_eymen@miuul.com,1985-01-23,39,463,(533)8082176,(538)4811026,36-45 YAŞ
3,88,Ayhan ÖZÇİL,3328037338,E,a_ozcil@miuul.com,1963-11-08,10,114,(543)2007274,(534)1062665,56-65 YAŞ
4,89,Azad ÖNÜR,65514513990,E,a_onvr@miuul.com,1989-03-23,55,722,(532)7551426,(542)2487070,20-35 YAŞ
5,109,Abdurrahman ALTINGÖZ,55960130916,E,a_altingoz@miuul.com,1999-08-07,37,720,(544)6667247,(538)2796190,20-35 YAŞ
6,155,Adil KINALI,62403300353,E,a_kinali@miuul.com,1995-04-29,21,675,(542)5082498,(533)8398360,20-35 YAŞ
7,202,Ayhan KARAMARLI,51329958677,E,a_karamarli@miuul.com,1957-12-02,39,598,(544)6372815,(532)5308665,65+ YAŞ
8,248,Azad ERGÜNEŞ,60171995213,E,a_ergvnes@miuul.com,1990-07-19,14,408,(538)3718436,(537)4538516,20-35 YAŞ
9,285,Azad KAYAR,12138400338,E,a_kayar@miuul.com,1981-03-22,37,174,(553)8521332,(555)7492984,36-45 YAŞ


#### SORU 8

In [8]:
# -- SORU 8
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE BIRTHDATE BETWEEN '1990-01-01' AND '1995-12-31'
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP
0,6,Ahmet İNCİKAPI,6722155596,E,a_incikapi@miuul.com,1991-05-28,53,225,(532)2414618,(538)8459085,20-35 YAŞ
1,8,Elif ÖZÇELİKBAŞ,84870496920,K,e_ozcelikbas@miuul.com,1993-06-06,73,815,(536)9014627,(544)3937372,20-35 YAŞ
2,13,Dilan DOKUYUCU,74659763913,K,d_dokuyucu@miuul.com,1993-01-21,25,333,(538)8929868,(534)4275461,20-35 YAŞ
3,14,Selim ÖZBAY,77720855989,E,s_ozbay@miuul.com,1992-10-02,73,815,(535)5906635,(533)4273519,20-35 YAŞ
4,30,Bülent KAÇAROĞLU,21971116249,E,b_kacaroglu@miuul.com,1995-01-09,42,311,(554)6844639,(541)5324664,20-35 YAŞ
...,...,...,...,...,...,...,...,...,...,...,...
95,978,Caner ÖZTEL,29584025693,E,c_oztel@miuul.com,1991-10-07,5,552,(543)3006551,(537)4588348,20-35 YAŞ
96,979,Turan AYSER,66777087518,E,t_ayser@miuul.com,1992-02-19,79,364,(535)7971479,(532)4299465,20-35 YAŞ
97,986,Ceylin COŞĞUN,17415297361,K,c_cosgun@miuul.com,1994-12-29,11,445,(533)8061812,(534)3459741,20-35 YAŞ
98,1012,Soner KALIPCI,49804660632,E,s_kalipci@miuul.com,1993-12-21,72,373,(537)8836191,(532)8253796,20-35 YAŞ


In [9]:
# -- SORU 8
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE YEAR(BIRTHDATE) BETWEEN 1990 AND 1995
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP
0,6,Ahmet İNCİKAPI,6722155596,E,a_incikapi@miuul.com,1991-05-28,53,225,(532)2414618,(538)8459085,20-35 YAŞ
1,8,Elif ÖZÇELİKBAŞ,84870496920,K,e_ozcelikbas@miuul.com,1993-06-06,73,815,(536)9014627,(544)3937372,20-35 YAŞ
2,13,Dilan DOKUYUCU,74659763913,K,d_dokuyucu@miuul.com,1993-01-21,25,333,(538)8929868,(534)4275461,20-35 YAŞ
3,14,Selim ÖZBAY,77720855989,E,s_ozbay@miuul.com,1992-10-02,73,815,(535)5906635,(533)4273519,20-35 YAŞ
4,30,Bülent KAÇAROĞLU,21971116249,E,b_kacaroglu@miuul.com,1995-01-09,42,311,(554)6844639,(541)5324664,20-35 YAŞ
...,...,...,...,...,...,...,...,...,...,...,...
95,978,Caner ÖZTEL,29584025693,E,c_oztel@miuul.com,1991-10-07,5,552,(543)3006551,(537)4588348,20-35 YAŞ
96,979,Turan AYSER,66777087518,E,t_ayser@miuul.com,1992-02-19,79,364,(535)7971479,(532)4299465,20-35 YAŞ
97,986,Ceylin COŞĞUN,17415297361,K,c_cosgun@miuul.com,1994-12-29,11,445,(533)8061812,(534)3459741,20-35 YAŞ
98,1012,Soner KALIPCI,49804660632,E,s_kalipci@miuul.com,1993-12-21,72,373,(537)8836191,(532)8253796,20-35 YAŞ


#### SORU 9

In [10]:
# -- SORU 9
df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
WHERE CT.CITY='İSTANBUL'
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP,ID,CITY
0,15,Yasin AĞAGÜL,32764684197,E,y_agagvl@miuul.com,1979-10-19,34,897,(532)6102663,(537)3381012,36-45 YAŞ,34,İSTANBUL
1,143,Sebahat CİLALITAŞ,65960134490,K,s_cilalitas@miuul.com,1978-09-30,34,64,(535)7019065,(532)2408341,36-45 YAŞ,34,İSTANBUL
2,152,Deniz BENDER,31619199155,E,d_bender@miuul.com,1986-04-04,34,134,(542)4181722,(536)4621320,36-45 YAŞ,34,İSTANBUL
3,156,Çağla BEĞEN,85581395736,K,c_begen@miuul.com,1991-12-22,34,81,(535)1338012,(533)8331511,20-35 YAŞ,34,İSTANBUL
4,182,Nurettin GAYRETLİ,2822523822,E,n_gayretli@miuul.com,1950-04-27,34,84,(532)7969080,(536)7322740,65+ YAŞ,34,İSTANBUL
5,195,Yeliz KÜÇÜKALP,65432369284,K,y_kvcvkalp@miuul.com,1965-02-26,34,488,(555)9613650,(543)1071715,56-65 YAŞ,34,İSTANBUL
6,222,Eylül GÜLÜ,56388773535,K,e_gvlv@miuul.com,1969-08-24,34,3,(537)5953916,(532)4647711,46-55 YAŞ,34,İSTANBUL
7,232,Müzeyyen OCAKÇI,89589164667,K,m_ocakci@miuul.com,1972-02-19,34,707,(543)1576124,(505)9181537,46-55 YAŞ,34,İSTANBUL
8,258,Muhammed Emin TEKKAYA,87271968026,E,m_emin@miuul.com,1984-04-19,34,897,(542)2911632,(535)9881318,36-45 YAŞ,34,İSTANBUL
9,267,Neslihan KILIÇÇEKER,53734331933,K,n_kilicceker@miuul.com,1982-10-04,34,707,(543)1432619,(505)2287257,36-45 YAŞ,34,İSTANBUL


#### SORU 10

In [11]:
# -- SORU 10
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE CITYID IN (
    SELECT ID
    FROM CITIES
    WHERE CITY='İSTANBUL')
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP
0,15,Yasin AĞAGÜL,32764684197,E,y_agagvl@miuul.com,1979-10-19,34,897,(532)6102663,(537)3381012,36-45 YAŞ
1,143,Sebahat CİLALITAŞ,65960134490,K,s_cilalitas@miuul.com,1978-09-30,34,64,(535)7019065,(532)2408341,36-45 YAŞ
2,152,Deniz BENDER,31619199155,E,d_bender@miuul.com,1986-04-04,34,134,(542)4181722,(536)4621320,36-45 YAŞ
3,156,Çağla BEĞEN,85581395736,K,c_begen@miuul.com,1991-12-22,34,81,(535)1338012,(533)8331511,20-35 YAŞ
4,182,Nurettin GAYRETLİ,2822523822,E,n_gayretli@miuul.com,1950-04-27,34,84,(532)7969080,(536)7322740,65+ YAŞ
5,195,Yeliz KÜÇÜKALP,65432369284,K,y_kvcvkalp@miuul.com,1965-02-26,34,488,(555)9613650,(543)1071715,56-65 YAŞ
6,222,Eylül GÜLÜ,56388773535,K,e_gvlv@miuul.com,1969-08-24,34,3,(537)5953916,(532)4647711,46-55 YAŞ
7,232,Müzeyyen OCAKÇI,89589164667,K,m_ocakci@miuul.com,1972-02-19,34,707,(543)1576124,(505)9181537,46-55 YAŞ
8,258,Muhammed Emin TEKKAYA,87271968026,E,m_emin@miuul.com,1984-04-19,34,897,(542)2911632,(535)9881318,36-45 YAŞ
9,267,Neslihan KILIÇÇEKER,53734331933,K,n_kilicceker@miuul.com,1982-10-04,34,707,(543)1432619,(505)2287257,36-45 YAŞ


#### SORU 11

In [12]:
# -- SORU 11
df = sorgu("""
SELECT 
    CT.CITY AS SEHIR,
    COUNT(*) AS [KISI SAYISI]
FROM CUSTOMERS AS C
RIGHT JOIN CITIES AS CT ON CT.ID=C.CITYID
GROUP BY CT.CITY
ORDER BY [KISI SAYISI] DESC
""")
df


,SEHIR,KISI SAYISI
0,ŞIRNAK,110
1,İSTANBUL,47
2,ANKARA,29
3,GAZİANTEP,25
4,İZMİR,24
...,...,...
76,KIRIKKALE,1
77,IĞDIR,1
78,KARAMAN,1
79,BAYBURT,1


In [13]:
# -- SORU 11
df = sorgu("""
SELECT 
    *,
    (SELECT COUNT(*)
    FROM CUSTOMERS
    WHERE CUSTOMERS.CITYID=CITIES.ID) AS [SAYI]
FROM CITIES
ORDER BY SAYI DESC
""")
df


,ID,CITY,SAYI
0,73,ŞIRNAK,110
1,34,İSTANBUL,47
2,6,ANKARA,29
3,27,GAZİANTEP,25
4,35,İZMİR,24
...,...,...,...
76,76,IĞDIR,1
77,69,BAYBURT,1
78,70,KARAMAN,1
79,71,KIRIKKALE,1


#### SORU 12

In [14]:
df = sorgu("""
SELECT
    CT.CITY,
    COUNT(*) AS [KULLANICI SAYISI]
FROM CUSTOMERS
JOIN CITIES AS CT ON CT.ID=CUSTOMERS.CITYID
GROUP BY CT.CITY
HAVING COUNT(*)>10
ORDER BY [KULLANICI SAYISI] DESC
""")
df


,CITY,KULLANICI SAYISI
0,ŞIRNAK,110
1,İSTANBUL,47
2,ANKARA,29
3,GAZİANTEP,25
4,İZMİR,24
5,KASTAMONU,23
6,BURSA,22
7,ÇORUM,22
8,DİYARBAKIR,20
9,BALIKESİR,20


In [15]:
df = sorgu("""
SELECT
    (SELECT CITY FROM CITIES WHERE ID=CUSTOMERS.CITYID),
    COUNT(*) AS [KULLANICI SAYISI]
FROM CUSTOMERS
GROUP BY CUSTOMERS.CITYID
HAVING COUNT(*)>10
ORDER BY [KULLANICI SAYISI] DESC
""")
df


,,KULLANICI SAYISI
0,ŞIRNAK,110
1,İSTANBUL,47
2,ANKARA,29
3,GAZİANTEP,25
4,İZMİR,24
5,KASTAMONU,23
6,BURSA,22
7,ÇORUM,22
8,BALIKESİR,20
9,DİYARBAKIR,20


#### SORU 13

In [16]:
df = sorgu("""
SELECT
    CT.CITY,
    GENDER, 
    COUNT(*) AS [KİŞİ SAYISI]
FROM CUSTOMERS
JOIN CITIES AS CT ON CT.ID=CITYID
GROUP BY 
    CT.CITY,
    GENDER
ORDER BY 
    1 ASC,
    3 DESC
""")
df


,CITY,GENDER,KİŞİ SAYISI
0,ADANA,E,10
1,ADANA,K,6
2,ADIYAMAN,E,7
3,ADIYAMAN,K,4
4,AFYONKARAHİSAR,K,13
...,...,...,...
145,YALOVA,K,1
146,YOZGAT,K,8
147,YOZGAT,E,5
148,ZONGULDAK,E,5


#### SORU 14

In [17]:
df = sorgu("""
SELECT
    CITY,
    (SELECT COUNT(*) FROM CUSTOMERS WHERE CITYID=C.CITYID) AS [MÜŞTERİ SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='K' AND CITYID=C.CITYID) AS [KADIN SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='E' AND CITYID=C.CITYID)  AS [ERKEK SAYISI]
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
GROUP BY CITYID, CITY
ORDER BY 2 DESC
""")
df


,CITY,MÜŞTERİ SAYISI,KADIN SAYISI,ERKEK SAYISI
0,ŞIRNAK,110,63,47
1,İSTANBUL,47,26,21
2,ANKARA,29,16,13
3,GAZİANTEP,25,11,14
4,İZMİR,24,14,10
...,...,...,...,...
75,DÜZCE,2,1,1
76,IĞDIR,1,0,1
77,BAYBURT,1,1,0
78,KARAMAN,1,1,0


In [18]:
df = sorgu("""
SELECT 
    CITY,
    (SELECT COUNT(*) FROM CUSTOMERS WHERE CT.ID=CITYID) AS [MÜŞTERİ SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='K' AND CT.ID=CITYID) AS [KADIN SAYISI],
    (SELECT COUNT(*) FROM CUSTOMERS WHERE GENDER='E' AND CT.ID=CITYID)  AS [ERKEK SAYISI]
FROM CITIES AS CT
ORDER BY 2 DESC
""")
df


,CITY,MÜŞTERİ SAYISI,KADIN SAYISI,ERKEK SAYISI
0,ŞIRNAK,110,63,47
1,İSTANBUL,47,26,21
2,ANKARA,29,16,13
3,GAZİANTEP,25,11,14
4,İZMİR,24,14,10
...,...,...,...,...
76,IĞDIR,1,0,1
77,BAYBURT,1,1,0
78,KARAMAN,1,1,0
79,KIRIKKALE,1,1,0


#### SORU 15

In [ ]:
df = sorgu("""
ALTER TABLE CUSTOMERS 
ADD AGEGROUP VARCHAR(50);
""")
df


#### SORU 16

In [ ]:
df = sorgu("""
UPDATE CUSTOMERS 
SET AGEGROUP='20-35 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 20 AND 35

UPDATE CUSTOMERS 
SET AGEGROUP='36-45 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 36 AND 45

UPDATE CUSTOMERS 
SET AGEGROUP='46-55 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 46 AND 55

UPDATE CUSTOMERS 
SET AGEGROUP='56-65 YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 56 AND 65

UPDATE CUSTOMERS 
SET AGEGROUP='65+ YAŞ' 
WHERE DATEDIFF(YEAR, BIRTHDATE, GETDATE()) > 65
""", select=False)
df


In [19]:
df = sorgu("""
SELECT *
FROM CUSTOMERS
WHERE AGEGROUP IS NULL
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP


#### SORU 17

In [20]:
df = sorgu("""
SELECT 
    ARALIK,
    COUNT(*) AS SAYI
FROM (
    SELECT 
        CASE
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 20 AND 35 THEN '20-35 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 36 AND 45 THEN '36-45 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 46 AND 55 THEN '46-55 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) BETWEEN 56 AND 65 THEN '56-65 YAŞ'
            WHEN DATEDIFF(YEAR, BIRTHDATE, GETDATE()) > 65 THEN '65+ YAŞ' 
        END AS ARALIK,
        *
    FROM CUSTOMERS) AS TEMP
GROUP BY ARALIK
""")
df


,ARALIK,SAYI
0,65+ YAŞ,266
1,46-55 YAŞ,163
2,20-35 YAŞ,207
3,36-45 YAŞ,154
4,56-65 YAŞ,161


#### SORU 18

In [21]:
df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
JOIN CITIES AS CT ON CT.ID=C.CITYID
JOIN DISTRICTS AS D ON D.ID=C.DISTRICT
WHERE 
    CT.CITY='İSTANBUL'
    AND D.DISTRICT<>'KADIKÖY'
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP,ID,CITY,ID,CITYID,DISTRICT
0,15,Yasin AĞAGÜL,32764684197,E,y_agagvl@miuul.com,1979-10-19,34,897,(532)6102663,(537)3381012,36-45 YAŞ,34,İSTANBUL,897,34,BAHÇELİEVLER
1,143,Sebahat CİLALITAŞ,65960134490,K,s_cilalitas@miuul.com,1978-09-30,34,64,(535)7019065,(532)2408341,36-45 YAŞ,34,İSTANBUL,64,80,BAHÇE
2,152,Deniz BENDER,31619199155,E,d_bender@miuul.com,1986-04-04,34,134,(542)4181722,(536)4621320,36-45 YAŞ,34,İSTANBUL,134,65,ÇATAK
3,156,Çağla BEĞEN,85581395736,K,c_begen@miuul.com,1991-12-22,34,81,(535)1338012,(533)8331511,20-35 YAŞ,34,İSTANBUL,81,2,BESNİ
4,182,Nurettin GAYRETLİ,2822523822,E,n_gayretli@miuul.com,1950-04-27,34,84,(532)7969080,(536)7322740,65+ YAŞ,34,İSTANBUL,84,34,BEYKOZ
5,195,Yeliz KÜÇÜKALP,65432369284,K,y_kvcvkalp@miuul.com,1965-02-26,34,488,(555)9613650,(543)1071715,56-65 YAŞ,34,İSTANBUL,488,54,SAPANCA
6,222,Eylül GÜLÜ,56388773535,K,e_gvlv@miuul.com,1969-08-24,34,3,(537)5953916,(532)4647711,46-55 YAŞ,34,İSTANBUL,3,34,ADALAR
7,232,Müzeyyen OCAKÇI,89589164667,K,m_ocakci@miuul.com,1972-02-19,34,707,(543)1576124,(505)9181537,46-55 YAŞ,34,İSTANBUL,707,42,KARATAY
8,258,Muhammed Emin TEKKAYA,87271968026,E,m_emin@miuul.com,1984-04-19,34,897,(542)2911632,(535)9881318,36-45 YAŞ,34,İSTANBUL,897,34,BAHÇELİEVLER
9,267,Neslihan KILIÇÇEKER,53734331933,K,n_kilicceker@miuul.com,1982-10-04,34,707,(543)1432619,(505)2287257,36-45 YAŞ,34,İSTANBUL,707,42,KARATAY


#### SORU 19

In [22]:
"""
DELETE FROM CITIES WHERE CITY='ANKARA'
DELETE FROM CITIES WHERE CITY='İSTANBUL'
"""

"""
SELECT *
FROM CITIES
"""

df = sorgu("""
SELECT *
FROM CUSTOMERS AS C
LEFT JOIN CITIES AS CT ON CT.ID=C.CITYID
WHERE CT.CITY IS NULL
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP,ID,CITY
0,167,Aslı GÜNE,63618747382,K,a_gvne@miuul.com,1994-05-06,6,157,(505)6442992,(532)8235020,20-35 YAŞ,None,None
1,190,Güllü SALUR,76464619181,K,g_salur@miuul.com,1998-01-24,6,128,(533)7408614,(535)5992618,20-35 YAŞ,None,None
2,217,Rukiye ÜNGÖR,1490087423,K,r_vngor@miuul.com,1956-07-07,6,756,(554)1519484,(538)7692165,65+ YAŞ,None,None
3,238,Güler BEŞKAYA,87951772201,K,g_beskaya@miuul.com,1998-07-21,6,756,(553)8215027,(555)8771930,20-35 YAŞ,None,None
4,246,Sefa CANGAR,17673520419,E,s_cangar@miuul.com,1958-07-31,6,361,(554)1434188,(533)3133445,56-65 YAŞ,None,None
5,268,İzzet CANKURU,76469454575,E,i_cankuru@miuul.com,1978-12-11,6,257,(555)3724345,(541)4935289,36-45 YAŞ,None,None
6,286,Seval ÖZKANLI,52599001986,K,s_ozkanli@miuul.com,1995-07-02,6,425,(534)7052964,(538)6732660,20-35 YAŞ,None,None
7,288,Halil İbrahim BİMBİRDİREK,12831119936,E,h_ibrahim@miuul.com,1971-07-19,6,756,(544)4642995,(537)9514174,46-55 YAŞ,None,None
8,295,Ecrin MÜRSEL,1693687461,K,e_mvrsel@miuul.com,1956-01-17,6,425,(535)3784675,(533)9489743,65+ YAŞ,None,None
9,427,Cihan SAYGINER,14412309644,E,c_sayginer@miuul.com,1943-03-26,6,30,(533)8984184,(537)3434486,65+ YAŞ,None,None


#### SORU 20

In [23]:
df = sorgu("""
SET IDENTITY_INSERT CITIES ON
INSERT INTO CITIES (ID, CITY) VALUES (6, 'ANKARA')
INSERT INTO CITIES (ID, CITY) VALEUS (34, 'İSTANBUL')
""")
df


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near 'VALEUS'. (102) (SQLExecDirectW)")

#### SORU 21

In [25]:
df = sorgu("""
SELECT 
    *,
    LEFT(TELNR1, 5) AS OPERATOR1,
    SUBSTRING(TELNR2, 2, 3) AS OPERATOR2
FROM CUSTOMERS
""")
df


,ID,NAMESURNAME,TCNUMBER,GENDER,EMAIL,BIRTHDATE,CITYID,DISTRICT,TELNR1,TELNR2,AGEGROUP,OPERATOR1,OPERATOR2
0,1,Sevda AKÇAN,42074151323,K,s_akcan@miuul.com,1964-05-12,22,202,(542)5255514,(532)3438190,56-65 YAŞ,(542),532
1,2,Sebahat ŞERALI,74789296130,K,s_serali@miuul.com,1943-06-11,78,473,(534)7157144,(532)2212126,65+ YAŞ,(534),532
2,3,Irmak HAMİDİ,86856513494,K,i_hamidi@miuul.com,1973-11-13,21,675,(533)2144819,(555)1183975,46-55 YAŞ,(533),555
3,4,Tuğçe AKKOÇ,50190579600,K,t_akkoc@miuul.com,1958-06-08,30,158,(543)4243115,(533)6299636,56-65 YAŞ,(543),533
4,5,Necdet ERÇAM,26046030220,E,n_ercam@miuul.com,1986-05-22,60,718,(532)6896237,(534)2924742,36-45 YAŞ,(532),534
...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,1037,Nisa nur MÜFTÜLÜĞÜ,20261366583,K,n_nur@miuul.com,1950-04-24,73,815,(537)4308013,(537)6345257,65+ YAŞ,(537),537
947,1038,Yusuf Eymen TARI,22465899881,E,y_eymen@miuul.com,1946-12-28,34,707,(542)8456299,(542)1368216,65+ YAŞ,(542),542
948,1040,Esma nur HIZLAR,36786286035,K,e_nur@miuul.com,1954-07-29,3,178,(537)4855581,(536)8432561,65+ YAŞ,(537),536
949,1041,Şaziye AKKOCA,58126168060,K,s_akkoca@miuul.com,1949-08-07,41,329,(541)5516258,(533)6463631,65+ YAŞ,(541),533
